In [39]:
import vectorbt as vbt
from vectorbt_ysj.common.constant import *

import pandas as pd
import numpy as np
from datetime import datetime
from sqlalchemy import create_engine
from urllib import parse
import plotly.io as pio

In [40]:

# 设置渲染器为浏览器
pio.renderers.default = "notebook"


In [47]:

def ma_test(symbols: list[str], start_date: datetime, end_date: datetime, interval: Interval):
    klineEngine = create_engine('mysql+pymysql://root:%s@localhost:3306/vnpy' % parse.quote_plus('admin'))
    query_kline_sql = "SELECT * FROM `dbbardata` WHERE symbol in ('%s') and `interval`='%s' and datetime>='%s' \
        and datetime<='%s' ORDER BY datetime;" % (','.join(symbols), interval.value, start_date.strftime('%Y-%m-%d %H:%M:%S'),
                                                  end_date.strftime('%Y-%m-%d %H:%M:%S'))
    klines = pd.read_sql_query(query_kline_sql, klineEngine)
    klines.set_index('datetime', inplace=True)
    if klines is not None and not klines.empty:
        # ma = vbt.MA.run(klines['close_price'], window=[20, 60], short_name='ma22')
        fast_ma = vbt.MA.run(klines['close_price'], window=20, short_name='fast_ma')
        slow_ma = vbt.MA.run(klines['close_price'], window=60, short_name='slow_ma')
        # print(f'ma={ma.ma.tail(10)}')
        # print(f'fast_ma={fast_ma.tail(10)}')
        # print(f'slow_ma={slow_ma.tail(10)}')
        long_open_signals = fast_ma.ma_crossed_above(slow_ma)
        long_close_signals = fast_ma.ma_crossed_below(slow_ma)
        short_open_signals = fast_ma.ma_crossed_below(slow_ma)
        short_close_signals = fast_ma.ma_crossed_above(slow_ma)
        portfolio = vbt.Portfolio.from_signals(
            close=klines['close_price'],
            entries=long_open_signals,
            exits=long_close_signals,
            short_entries=short_open_signals,
            short_exits=short_close_signals,
            init_cash=60000,
            freq='D',
            size=10,
            fees=0.0002,
            slippage=0.0003
        )
        print(portfolio.stats())
        portfolio.plot().show()

In [49]:


if __name__ == "__main__":
    """"""
    t0 = datetime.now()

    startDate = datetime(2024, 1, 1, 9, 0, 0)
    endDate = datetime(2025, 5, 14, 15, 0, 0)
    ma_test(['RBL9', 'SAL9'], startDate, endDate, Interval.DAILY)

    t1 = datetime.now()
    print(f'\n>>>>>>总耗时{t1 - t0}s')

KeyboardInterrupt: 